## **Setup**

In [ ]:
!pip install "tensorflow<1.14"
!pip install "tensorflow-gpu<1.14" 
!apt-get install libsndfile1
!pip install openl3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
  Using cached https://files.pythonhosted.org/packages/e2/88/8536723b81c47ada614d008a75e71923932f22e35ab89d4bf5fe441b58fc/openl3-0.3.1.tar.gz
     |████████████████████████████████| 337kB 2.8MB/s 
     |████████████████████████████████| 389kB 12.2MB/s 
     |████████████████████████████████| 25.4MB 95kB/s 
     |████████████████████████████████| 3.3MB 37.5MB/s 
     |████████████████████████████████| 26.9MB 113kB/s 
  Created wheel for openl3: filename=openl3-0.3.1-py2.py3-none-any.whl size=249323247 sha256=4e145499af986341125f428b1e1e6c4b3228176639d6671e4b143f2b986d55f6
  Stored in directory: /root/.cache/pip/wheels/13/63/c9/35868f3dd3b466909e73178db8566430da2d093cc055b932b1
  Created wheel for moviepy: filename=moviepy-1.0.3-cp36-none-any.whl size=110729 sha2

In [ ]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import sys

from tensorflow.keras import backend as K
# from tensorflow.keras.models import Sequential, load_model, Model
# from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Input
# from tensorflow.keras.losses import BinaryCrossentropy
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.metrics import AUC
#from tensorflow.keras.callbacks import EarlyStopping


from sklearn.metrics import roc_auc_score, roc_curve, auc
from matplotlib.colors import LinearSegmentedColormap
from termcolor import colored

# from tensorflow.keras.applications import ResNet50, VGG19, InceptionResNetV2
# from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess_input
# from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input
# from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as inception_resnet_v2_preprocess_input
# from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Set seed for reproducibility of results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

tf.compat.v1.reset_default_graph()

random.seed(seed_value)
np.random.seed(seed_value)
#tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)

# Configure a new global `tensorflow` session
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

tf.__version__

'1.13.2'

In [ ]:
from google.colab import drive
mount_path = '/content/gdrive/'
drive.mount(mount_path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
# Set and test path to competition data files
competition_path = 'My Drive/Final Project ITC/MAFAT Challenge/Data'
try:
  if competition_path == 'INSERT HERE':
    print('Please enter path to competition data files:')
    competition_path = input()
  file_path = 'MAFAT RADAR Challenge - Training Set V1.csv'
  with open(f'{mount_path}/{competition_path}/{file_path}') as f:
    f.readlines()
  print(colored('Everything is setup correctly', color='green'))
except:
  print(colored('Please mount drive and set competition_path correctly',
                color='red'))

Everything is setup correctly


In [ ]:
model_input = {
                'data_extraction':  {
                                    'train_df': [2, False],  # [num segments in track, balance the data?]
                                    'valid_df': [2, True],
                                    'exp_df':   [1, False],
                                    'synth_df': [1, True]
                                    },
                'model_conf' : {
                              'sota_model': "ResNet50",
                              'sota_model_layer': "conv4_block1_2_conv",
                              'input_last_channel': 3,
                              'batch_norm': False,
                              'batch_size':64,
                              'no_epochs':100,
                              'learning_rate': 0.0005,
                              'monitor': 'val_accuracy'
                              },
               'callback' : {'early_stop':{
                                           'patience':10,
                                           'verbose': 1
                                          },
                            'model_checkpoint':{
                                                'flag':True,
                                                'file_path':f"{mount_path}My Drive/Final Project ITC/MAFAT Challenge/Our notebooks/Inbar/saved_models",
                                                'save_weights_only':False,
                                                'verbose': 1
                                                }
                            },
               'IDG' : {
                        'vertical_flip':True,
                        'horizontal_flip':True,
                        'width_shift_range' : 0,
                        'height_shift_range': 0,
                        'fill_mode':'nearest',
                        'brightness_range' : [1,1],
                        'preprocessing_function' : None
                        }        
                }

## **Functions**

All functions will be used in the "Training the Model" section.

In [ ]:
# paths for the datasets
experiment_auxiliary_path = 'MAFAT RADAR Challenge - Auxiliary Experiment Set V2'
synthetic_auxiliary_path = 'MAFAT RADAR Challenge - Auxiliary Synthetic Set V2'
train_path = 'MAFAT RADAR Challenge - Training Set V1'
test_path = 'MAFAT RADAR Challenge - Public Test Set V1'

In [ ]:
files_path = mount_path + "/" + "My Drive/Final Project ITC/MAFAT Challenge/Our notebooks/python_scripts"

sys.path.append(files_path)

In [ ]:
from loading_functions import load_data, load_pkl_data, load_csv_metadata, load_data_all_datasets, append_dict
from spectogram_plots import plot_shited_spectrogram, plot_recenter_midline_spectrogram, spectrogram_cmap
from model_roc_function import stats
from data_augmentation import  recenter_midline, shift_spectrogram
from data_preprocessing_funcs import data_preprocess
from plot_learning_curve import plot_loss_and_accuracy_over_epoches
from data_augmentation import recenter_midline, shift_spectrogram
from full_loading_and_process import process_and_split_data
from sampling_data import subsampling, subsampling_segments_target_ratio, balance_target, split_x_y, split_train_val


np.save('/content/cmap.npy', spectrogram_cmap)

### **Splitting the Training set**

The functions below split the training set into Train and Validation sets.

* Validation Set.   
The validation set is constructed from segments from geolocation_ids 1 and 4.   
These geolocations' target_type distributions are balanced.
To create a fair representation of the test set   
(one segment form each track) every sixth segment was taken to the validation set.   
The reason is that approximately 75% of the tracks have less than 6 segments,   
it guarantees that most of the segments in the validation set are not from the same track.   
   
* Adding "human" segments to the training set.   
The training set is imbalanced.   
One can address this issue by taking segements of humans from the "Auxiliary Experiment Set".   
The first 3 segments from each track are taken into the training set (or less if the track is shorter than 3 segments).

In [ ]:
def load_and_split_data():
  training_df, synthetic_auxiliary_df, experiment_auxiliary_df = load_data_all_datasets()
  train_x, val_x, test_x, train_y, val_y, test_df  = process_and_split_data(training_df, synthetic_auxiliary_df, experiment_auxiliary_df, model_input['data_extraction'])
  del training_df, synthetic_auxiliary_df, experiment_auxiliary_df
  return train_x, val_x, test_x, train_y, val_y, test_df

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_df  = load_and_split_data()

[INFO] Loading and spliting the data
[INFO] Loading Auxiliary Experiment set - can take a few minutes
[INFO] Loading Auxiliary Synthetic set - can take a few minutes
[INFO] Loading Train set - can take a few minutes
exp_df 1 False
synt_df 1 True
train_df 2 False
valid_df 2 True
exp_df humans segments: 2718 0
synt_df humans segments: 447 447
train_df humans segments: 0 1972
valid_df humans segments: 223 223
[INFO] Adding segments from the experiment and Synthetic auxiliary sets to the training set
[INFO] Preprocessing and split the data to training and validation
[INFO] Creating 3 channels for the train and validation set
[INFO] Loading and preprocessing public test set


In [ ]:
train_x.shape

(5584, 126, 32, 3)

In [ ]:
train_x = train_x[:, :, :, 1]
train_x = train_x.reshape(list(train_x.shape)+[1])

# val_x = val_x[:, :, :, 1]
# val_x = val_x.reshape(list(val_x.shape)+[1])

# test_x = test_x[:, :, :, 1]
# test_x = test_x.reshape(list(test_x.shape)+[1])

# train_x.shape, val_x.shape, test_x.shape

In [ ]:
train_x.shape

(5584, 126, 32, 1)

## **Model**

After 

### **CNN Model** - transfer learning


In [ ]:
# Building the model
def create_model(model_tl, init, layer, input_shape= (126, 32, 3)):
  """
  Transfer learning Resnet model. trained on imagenet
  The input shape must be RGB, hense we must duplicate the channel 3 times

  Arguments:
    input_shape -- the shape of our input
    init -- the weight initialization

  Returns:
    CNN model - with resnet inside  
  """
  tl_model = Sequential(name="transfer_learning_model")
  tl_model.add(Input(shape=input_shape))

  transfer_learning_model = model_tl(include_top=False, weights="imagenet", pooling='avg')

  tl_model.add(Model(inputs=transfer_learning_model.input, outputs=transfer_learning_model.get_layer(layer).output))
  tl_model.add(Flatten())
  
  tl_model.add(Dense(1, activation='sigmoid', kernel_initializer = init))

  # Say not to train first layer (ResNet) model. It is already trained
  tl_model.layers[0].trainable = False
  return tl_model

## **Training The Model**

#### loading the data and pre-processing

### embeding for training set

In [ ]:
import openl3
from skimage.io import imread
from scipy import interpolate
from tqdm.notebook import tqdm

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [ ]:
def embedding_preprocess(batch_seg):
  
  segments = []
  for segment in batch_seg:
    spec_iq = recenter_midline(segment)

    # interporlate it to reach the 224x224 requirement for the L3 embedding 
    # so first interpolate to double its size and then repeate it 4 times
    X_o = np.linspace(1,spec_iq.shape[0],spec_iq.shape[0])
    Y_o = np.linspace(1,spec_iq.shape[1],spec_iq.shape[1])
    X_d = np.linspace(1,spec_iq.shape[0],2*spec_iq.shape[0])
    Y_d = np.linspace(1,spec_iq.shape[1],2*spec_iq.shape[1])
    f = interpolate.RectBivariateSpline(X_o,Y_o,spec_iq)
    spec_double = f(X_d,Y_d)

    spec_double_4 = np.concatenate([spec_double] * 4,axis=1)
    segments.append(spec_double_4)

  segments = np.array(segments)
  input = np.repeat(segments[...,np.newaxis], 3, -1)

  return input


In [ ]:
group_size = 128

In [ ]:
train_gen = (train_x[i:i + group_size] for i in np.arange(0, len(train_x), group_size))

In [ ]:
emb_env_list, emb_mus_list = [], []

In [ ]:
for batch_segs in tqdm(train_gen, total=len(train_x)//group_size):
  seg_emb = embedding_preprocess(batch_segs)  
  emb_env = openl3.get_image_embedding(seg_emb, content_type="env",
                                  input_repr="mel256", batch_size=64)

  emb_mus = openl3.get_image_embedding(seg_emb, content_type="music",
                                  input_repr="mel256", batch_size=64)
  
  emb_env_list.append(emb_env)
  emb_mus_list.append(emb_mus)

80/80 [==============================] - 64s 805ms/step



In [ ]:
emb_env_list[0].shape

(128, 8192)

In [ ]:
len(emb_env_list)

44

In [ ]:
len(emb_mus_list)

44

### embeding for validation set

In [ ]:
val_x = val_x[:, :, :, 1]
val_x = val_x.reshape(list(val_x.shape)+[1])

In [ ]:
group_size = 32
val_gen = (val_x[i:i + group_size] for i in np.arange(0, len(val_x), group_size))

In [ ]:
emb_env_list_val, emb_mus_list_val = [], []

In [ ]:
for batch_segs in tqdm(val_gen, total=len(val_x)//group_size):
  seg_emb = embedding_preprocess(batch_segs)  
  emb_env = openl3.get_image_embedding(seg_emb, content_type="env",
                                  input_repr="mel256", batch_size=group_size // 2)

  emb_mus = openl3.get_image_embedding(seg_emb, content_type="music",
                                  input_repr="mel256", batch_size=group_size // 2)
  
  emb_env_list_val.append(emb_env)
  emb_env_list_val.append(emb_mus)

30/30 [==============================] - 76s 3s/step



In [ ]:
emb_env_list_val[0].shape

(32, 8192)

In [ ]:
emb_mus_list_val = [emb_env_list_val[i] for i in range(len(emb_env_list_val)) if i % 2 !=0]

In [ ]:
emb_env_list_val = [emb_env_list_val[i] for i in range(len(emb_env_list_val)) if i % 2 ==0]

In [ ]:
len(emb_mus_list_val)

14

In [ ]:
len(emb_env_list_val)

14

### embeding for test set

In [ ]:
test_x = test_x[:, :, :, 1]
test_x = test_x.reshape(list(test_x.shape)+[1])

In [ ]:
test_x.shape

(106, 126, 32, 1)

In [ ]:
group_size = 2
test_gen = (test_x[i:i + group_size] for i in np.arange(0, len(test_x), group_size))

In [ ]:
emb_env_list_test, emb_mus_list_test = [], []

In [ ]:
for batch_segs in tqdm(test_gen, total=len(test_x)//group_size):
  seg_emb = embedding_preprocess(batch_segs)  
  emb_env = openl3.get_image_embedding(seg_emb, content_type="env",
                                  input_repr="mel256", batch_size=group_size )

  emb_mus = openl3.get_image_embedding(seg_emb, content_type="music",
                                  input_repr="mel256", batch_size=group_size)
  
  emb_env_list_test.append(emb_env)
  emb_mus_list_test.append(emb_mus)

2/2 [==============================] - 22s 11s/step



In [ ]:
embeding_test = {"x_emb_test": {"evn": emb_env_list_test, "mus": emb_mus_list_test},
                 "x_test": test_x,
                 "test_df": test_df
                }

In [ ]:
files_path = mount_path + "/" + "My Drive/Final Project ITC/MAFAT Challenge/Our notebooks/saved_models"

In [ ]:
with open(files_path + '/embeding_inbar_test_data.pickle', 'wb') as handle:
    pickle.dump(embeding_test, handle)

### save the results in a pickle file

In [ ]:
embeding = {"x_emb_train": {"evn": emb_env_list, "mus": emb_mus_list},
            "x_emb_val": {"evn": emb_env_list_val, "mus": emb_mus_list_val}, 
            "
            "train_y": train_y,
            "val_y": val_y,
            "data_extraction": model_input['data_extraction'],
            "x_test": test_x,
            "test_df": test_df
            }

In [ ]:
files_path + '/embeding_inbar_30_9.pickle'

'/content/gdrive//My Drive/Final Project ITC/MAFAT Challenge/Our notebooks/Inbar/embeding_inbar_30_9.pickle'

In [ ]:
with open(files_path + '/embeding_inbar_30_9.pickle', 'wb') as handle:
    pickle.dump(embeding, handle)

In [ ]:
with open(files_path + '/embeding_inbar_30_9.pickle', 'rb') as handle:
    b = pickle.load(handle)


In [ ]:
b.keys()

dict_keys(['x_emb_train', 'x_emb_val', 'train_y', 'val_y', 'data_extraction', 'x_test', 'test_df'])